<a href="https://colab.research.google.com/github/muriloufu/Hydrocyclone_ANN/blob/main/Vieira_2006_HC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from google.colab import files
import numpy as np
import io
#nao foi necessario a funcao do numpy
#o 'pd' vai ser usado para chamar o dataframe
#o 'plt' vai ser chamado para montar o grafico que relaciona valor observado (experimento) e valor predito pela rede neural
#o 'files' chama o arquivo do computador
#MLPRegressor é o comando da rede neural
#meam_squared_error (MSE) e o r2_score sao parametros que auxiliam a julgar a qualidade da regressao que serah feita pela rede neural
#Objetivo: A partir de um dataset de 39 experimentos ajustar uma resposta de interesse [flux improvment efficiency (FIE)]
#A funcao de ativaçao utilizada serah a tangenente hiperbolica (logistic)
#As inputs: X1 (inlet velocity)
#           X2 (Concentration)
#           X3 (TMP - Transmebrane Pressure)
#A output: FIE

In [9]:
#selecao do dataset
from google.colab import files
uploaded=files.upload()

Saving Tese - LG - 2006 - HCON.xlsx to Tese - LG - 2006 - HCON (1).xlsx


In [10]:
#colocar todo o dataset em um dataframe
df=pd.read_excel(io.BytesIO(uploaded['Tese - LG - 2006 - HCON.xlsx']))
df

,Di (mm),Do (mm),L (mm),θ (°),RL,Eu,η
0,4.8,6.6,142,11.2,28.28,4493,65.18
1,4.8,6.6,142,17.8,29.30,4407,61.76
2,4.8,6.6,207,11.2,31.80,3997,72.47
3,4.8,6.6,207,17.8,31.20,3664,62.23
4,4.8,9.6,142,11.2,10.24,3697,54.06
5,4.8,9.6,142,17.8,10.50,3211,47.30
6,4.8,9.6,207,11.2,15.04,3257,61.52
7,4.8,9.6,207,17.8,17.16,3016,54.30
8,7.8,6.6,142,11.2,29.00,2416,74.59
9,7.8,6.6,142,17.8,33.00,1789,69.00


In [16]:
#Retirada da coluna 0 do df e colocar no Dat_Select
Index=[0,1,2,3,4,5,6]
Dat_Select=[]
Dat_Select=df.iloc[:,Index].copy()
#Mosta o Dat_Select
Dat_Select

,Di (mm),Do (mm),L (mm),θ (°),RL,Eu,η
0,4.8,6.6,142,11.2,28.28,4493,65.18
1,4.8,6.6,142,17.8,29.30,4407,61.76
2,4.8,6.6,207,11.2,31.80,3997,72.47
3,4.8,6.6,207,17.8,31.20,3664,62.23
4,4.8,9.6,142,11.2,10.24,3697,54.06
5,4.8,9.6,142,17.8,10.50,3211,47.30
6,4.8,9.6,207,11.2,15.04,3257,61.52
7,4.8,9.6,207,17.8,17.16,3016,54.30
8,7.8,6.6,142,11.2,29.00,2416,74.59
9,7.8,6.6,142,17.8,33.00,1789,69.00


In [17]:
# funcao normatiza dados - FUNCAO RETIRADA DO ARTIGO
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        x[i]=2*((x[i]-min_x)/(max_x-min_x))-1
    return x

In [18]:
# funcao retorna os dados a forma original - xi ISOLADO DA FUNCAO: NORMATIZA(X)
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        x[i]=((x[i]+1)/2)*(max_x-min_x)+min_x
    return x

In [19]:
# separa os dados em normatizados e originais
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)
#Mosta o Dat_Select normatizado
Dat_Select.head(39)

,Di (mm),Do (mm),L (mm),θ (°),RL,Eu,η
0,-6.250000e-01,-6.250000e-01,-0.549550,-0.6,0.117028,0.285241,0.215087
1,-6.250000e-01,-6.250000e-01,-0.549550,0.6,0.180186,0.253589,-0.017329
2,-6.250000e-01,-6.250000e-01,0.621622,-0.6,0.334985,0.102687,0.710499
3,-6.250000e-01,-6.250000e-01,0.621622,0.6,0.297833,-0.019875,0.014611
4,-6.250000e-01,6.250000e-01,-0.549550,-0.6,-1.000000,-0.007729,-0.540605
5,-6.250000e-01,6.250000e-01,-0.549550,0.6,-0.983901,-0.186603,-1.000000
6,-6.250000e-01,6.250000e-01,0.621622,-0.6,-0.702786,-0.169672,-0.033639
7,-6.250000e-01,6.250000e-01,0.621622,0.6,-0.571517,-0.258373,-0.524295
8,6.250000e-01,-6.250000e-01,-0.549550,-0.6,0.161610,-0.479205,0.854570
9,6.250000e-01,-6.250000e-01,-0.549550,0.6,0.409288,-0.709974,0.474686


In [ ]:
#separa os dados em treino e teste -> Aqui os dados que eu coloquei no treino e teste foram identicos aos dados usados pelos autores
train=DAT.iloc[0:32,:]
test=DAT.iloc[32:39:]

In [ ]:
#obtem as variaveis independentes para o treino e para o teste
x_train=train.iloc[:,[0,1,2]]
x_test=test.iloc[:,[0,1,2]]
X_OLD=DAT_OLD.iloc[:,[0,1,2]]
#mostra as variaveis independentes do train (treino)
print(x_train)

In [ ]:
#obtem a variavel dependentes do treino e teste (FIE)
Index=[3]
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]
#Mostra a variavel dependente do train
print(y_train)

In [ ]:
#mostrar o X_test
print(x_test)

In [ ]:
#mostrar o y_test
print (y_test)

In [ ]:
#define a rede neural
#nessa rede vao ser usados 3 camadas: Input + hidden + output
#o numero de neuronios da output eh 1, porque tem 1 resposta (FIE)
#o numero de neuronios na hidden eh arbitrario. O artigo fez 12 neuronios nessa camada
#funcao de ativicao eh a logistic, segundo o artigo
model = MLPRegressor(solver='lbfgs',activation='logistic',alpha=1e-5,hidden_layer_sizes=(12),
                    random_state=1, max_iter=1000)
#o solver eh o lbfgs usado para metodos quasi-newton
#coloquei 1000 itercoes

In [ ]:
#treina a rede neural
model.fit(x_train, y_train)

In [ ]:
#usa a rede neural para treino e teste
y_calc_train=model.predict(x_train)
y_calc_test=model.predict(x_test)

In [ ]:
#transforma a saida da rede em dataframe 
y_calc_train=pd.DataFrame(y_calc_train)
y_calc_test=pd.DataFrame(y_calc_test)
col_names=list(y_train)
y_calc_train.columns = col_names
y_calc_test.columns = col_names

In [ ]:
#Desnormatizar os dados obtidos da rede neural
y_calc_train=Original(y_calc_train,Y_OLD)
y_calc_test=Original(y_calc_test,Y_OLD)

In [ ]:
#recupera os dados originais 
test=[]
train=[]
train=Original(y_train,Y_OLD)
test=Original(y_test,Y_OLD)

In [ ]:
#Resposta para fazer o grafico (FIE-treino)
resist_obs_train=[]
resist_calc_train=[]
resist_calc_train=y_calc_train['Y'].copy()
resist_obs_train=train['Y'].copy()

In [ ]:
#Resposta para fazer o grafico (FIE-teste)
resist_obs_test=[]
resist_calc_test=[]
resist_calc_test=y_calc_test['Y'].copy()
resist_obs_test=test['Y'].copy()

In [ ]:
#expressa os dois dados em figura - EULER
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(resist_obs_test,resist_calc_test, s=10, c='b', marker="s", label='teste')
ax1.scatter(resist_obs_train,resist_calc_train, s=10, c='r', marker="o", label='treino')
plt.legend(loc='upper left')
plt.ylabel('FIE Calc')
plt.xlabel('FIE Obs')
plt.show()

In [ ]:
#checar a qualidade da regressao PARA TESTE
mse=mean_squared_error(resist_obs_test,resist_calc_test)
print("MSE teste=",mse)
R2=r2_score(resist_obs_test,resist_calc_test)
print("R^2 teste=",R2)

In [ ]:
#checar a qualidade da regressao PARA TREINO
mse=mean_squared_error(resist_obs_train,resist_calc_train)
print("MSE treino=",mse)
R2=r2_score(resist_obs_train,resist_calc_train)
print("R^2 treino=",R2)